# QA on a corpus of PDF files.

> Building a simple chatbot to allow the user to ask questions about a collection of PDF files (e.g., grant proposals). 

In [1]:
#| default_exp tools.tiab_corpus_qa

In [2]:
#| export

import os
import pandas as pd

import re
import os

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index import Document, ServiceContext, set_global_service_context
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.node_parser import SentenceWindowNodeParser
    
from alhazen.core import get_llamaindex_llm

from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)

import gradio as gr
import os
from tqdm import tqdm

from pathlib import Path
from llama_index import download_loader

import fire

/Users/gburns/miniconda3/envs/tmp_llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#| export 

from llama_index import download_loader
PyMuPDFReader = download_loader("PyMuPDFReader")

class FileCollectionChatBot:

    def __init__(self, 
                doc_dir, 
                llm_name, 
                embed_model_name="BAAI/bge-small-en-v1.5", 
                sentence_window_size=10):

        self.embed_model = HuggingFaceEmbedding(model_name=embed_model_name)
        
        if doc_dir[-1:] != '/':
            doc_dir += '/'
        self.doc_dir = doc_dir

        self.llm = get_llamaindex_llm(llm_name)     

        # create the sentence window node parser w/ default settings
        self.node_parser = SentenceWindowNodeParser.from_defaults(
            window_size=sentence_window_size,
            window_metadata_key="window",
            original_text_metadata_key="original_text",
            )
        self.embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

        # create a service context
        self.service_context = ServiceContext.from_defaults(
            llm=self.llm,
            embed_model=self.embed_model,
            node_parser=self.node_parser)
        
        # List all pdf files in doc_dir by walking the whole directory tree
        pdf_file_list = []
        for root, dirs, files in os.walk(self.doc_dir):
            for file in files:
                if file.endswith('.pdf'):
                    pdf_file_list.append(os.path.join(root, file))
        
        self.loader = PyMuPDFReader()
        
        # Load each PDF block as a document
        documents = []
        print('Loading Documents...')
        for file in tqdm(pdf_file_list):
            pdf_documents = self.loader.load_data(file_path=file, metadata=True)
            documents.extend(pdf_documents)
        print('Loading Complete!')

        self.index = VectorStoreIndex.from_documents(documents, service_context=self.service_context)
        self.query_engine = self.index.as_query_engine()
    
    def run_gradio(self):

        def add_text(history, text):
            #print('add_text: history: %s, text: %s'%(history, text))
            history = history + [(text, None)]
            return history, gr.Textbox(value="", interactive=False)

        def add_file(history, file):
            #print('add_history: history: %s, file: %s'%(history, file))
            history = history + [((file.name,), None)]
            return history

        def bot(history):
            #print('bot: history: %s'%(history))
            # prompt to send to the agent is the last message from the user
            prompt = history[-1][0]
            response = self.query_engine.query(prompt)
            print('RESPONSE: %s'%(str(response)))
            history[-1][1] = str(response)
            print('WHOLE HISTORY: %s'%(history))
            return history

        with gr.Blocks() as demo:
            chatbot = gr.Chatbot(
                [],
                elem_id="chatbot",
                bubble_full_width=False,
                #avatar_images=(None, files(alhazen_resources).joinpath('alhazen.png'))
            )
            with gr.Row():
                txt = gr.Textbox(
                    scale=4,
                    show_label=False,
                    placeholder="Enter text and press enter, or upload files",
                    container=False,
                )
                btn = gr.UploadButton("📁", file_types=["image", "video", "audio"])

            txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(bot, chatbot, chatbot)
            txt_msg.then(lambda: gr.Textbox(interactive=True), None, [txt], queue=False)
            
            file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False).then(bot, chatbot, chatbot)

        demo.queue()
        demo.launch()


In [11]:
def pdf_corpus_chatbot(doc_dir, 
        llm_name, 
        embed_model_name="BAAI/bge-small-en-v1.5", 
        sentence_window_size=3):
    
    chatbot = FileCollectionChatBot(doc_dir, 
                llm_name,
                embed_model_name=embed_model_name, 
                sentence_window_size=sentence_window_size)
    chatbot.run_gradio()

#os.environ['LLMS_TEMP_DIR'] = '/tmp/alhazen'
#pdf_corpus_chatbot('/Users/gburns/Documents/2023H2/makeathon/grant_analysis', 'llama-2-70b-chat')

#os.environ['OPENAI_API_KEY'] = '<add your key here>'
#pdf_corpus_chatbot('/Users/gburns/Documents/2023H2/makeathon/grant_analysis', 'gpt-3.5')